##  Five Day Weather Forecast Information

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime

In [2]:
weather_forecast = requests.get("https://pro.openweathermap.org/data/2.5/forecast/hourly?q=Dublin&appid=9a8ddf0c4ecf94975ce6a1c702afbe2d")
fiveDayJSON = weather_forecast.json()
fiveDayJSON

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1624899600,
   'main': {'temp': 291.3,
    'feels_like': 291.05,
    'temp_min': 291.3,
    'temp_max': 295.72,
    'pressure': 1010,
    'sea_level': 1010,
    'grnd_level': 996,
    'humidity': 72,
    'temp_kf': -4.42},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 8},
   'wind': {'speed': 2.3, 'deg': 247, 'gust': 2.88},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2021-06-28 17:00:00'},
  {'dt': 1624903200,
   'main': {'temp': 293.46,
    'feels_like': 293.19,
    'temp_min': 293.46,
    'temp_max': 298.34,
    'pressure': 1010,
    'sea_level': 1010,
    'grnd_level': 996,
    'humidity': 63,
    'temp_kf': -4.88},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 12},
   'wind': {'speed': 2.74, 'deg': 250, 'gust': 3.19},
   'visibility': 1

## Normalise the JSON file

In [44]:
fiveDayForecast = pd.json_normalize(fiveDayJSON, record_path =['list'])
fiveDayForecast.head()

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod
0,1624888800,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0,2021-06-28 14:00:00,286.77,286.51,286.77,287.87,1008,1008,996,89,-1.10,83,2.35,240,3.51,d
1,1624892400,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0,2021-06-28 15:00:00,287.76,287.50,287.76,289.67,1008,1008,997,85,-1.91,75,2.22,239,3.14,d
2,1624896000,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0,2021-06-28 16:00:00,290.11,289.82,290.11,292.53,1009,1009,997,75,-2.42,62,2.20,240,2.97,d
3,1624899600,"[{'id': 802, 'main': 'Clouds', 'description': ...",10000,0,2021-06-28 17:00:00,293.87,293.54,293.87,295.72,1009,1009,996,59,-1.85,46,2.30,247,2.88,d
4,1624903200,"[{'id': 802, 'main': 'Clouds', 'description': ...",10000,0,2021-06-28 18:00:00,298.34,298.04,298.34,298.34,1009,1009,996,43,0.00,29,2.74,250,3.19,d


In [45]:
forcast_type = fiveDayForecast[['pop','main.temp','wind.speed', 'main.humidity', 'dt']]
forcast_type.head()

,pop,main.temp,wind.speed,main.humidity,dt
0,0,286.77,2.35,89,1624888800
1,0,287.76,2.22,85,1624892400
2,0,290.11,2.20,75,1624896000
3,0,293.87,2.30,59,1624899600
4,0,298.34,2.74,43,1624903200


In [46]:
forcast_fiveday = pd.json_normalize(fiveDayJSON, record_path = ['list', 'weather'])[['main','description']]
forcast_fiveday.head()

,main,description
0,Clouds,broken clouds
1,Clouds,broken clouds
2,Clouds,broken clouds
3,Clouds,scattered clouds
4,Clouds,scattered clouds


In [47]:
fiveDayForecast = pd.concat([forcast_fiveday, forcast_type], axis=1)
fiveDayForecast.head()

,main,description,pop,main.temp,wind.speed,main.humidity,dt
0,Clouds,broken clouds,0,286.77,2.35,89,1624888800
1,Clouds,broken clouds,0,287.76,2.22,85,1624892400
2,Clouds,broken clouds,0,290.11,2.20,75,1624896000
3,Clouds,scattered clouds,0,293.87,2.30,59,1624899600
4,Clouds,scattered clouds,0,298.34,2.74,43,1624903200


In [49]:
fiveDayForecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   main           96 non-null     object 
 1   description    96 non-null     object 
 2   pop            96 non-null     int64  
 3   main.temp      96 non-null     float64
 4   wind.speed     96 non-null     float64
 5   main.humidity  96 non-null     int64  
 6   dt             96 non-null     int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 5.4+ KB


## Finalised Dataferame

In [50]:
fiveDayForecast['dt'] = pd.to_datetime(fiveDayForecast['dt'], unit='s')
fiveDayForecast = fiveDayForecast.rename(columns={"main.temp": "temp", "wind.speed": "wind_speed", "main.humidity": "humidity"})
fiveDayForecast.head()

,main,description,pop,temp,wind_speed,humidity,dt
0,Clouds,broken clouds,0,286.77,2.35,89,2021-06-28 14:00:00
1,Clouds,broken clouds,0,287.76,2.22,85,2021-06-28 15:00:00
2,Clouds,broken clouds,0,290.11,2.20,75,2021-06-28 16:00:00
3,Clouds,scattered clouds,0,293.87,2.30,59,2021-06-28 17:00:00
4,Clouds,scattered clouds,0,298.34,2.74,43,2021-06-28 18:00:00
